# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
Vacation_data= "../VacationPY/output_data/cities.csv"
vacationData=pd.read_csv(Vacation_data)

vacationData.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
#Use the Lat and Lng as locations and Humidity as the weight.

locations = vacationData[["Lat", "Lng"]].astype(float)

#humidity
humidity = vacationData["Humidity"].astype(float)

#dimensions for graph
figure_layout = {
     'width' : '1000px',
     'height':'400px',
     'border':'1px solid black',
     'padding':'1px',
     'margin':'0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout)

fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=100,
                                   point_radius=5)
fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
temperatureCriteria = vacationData['Max Temp'] < 75
temperatureCriteria = vacationData['Max Temp'] > 55
windSpeedCriteria = vacationData['Wind Speed'] < 10
cloudinessCriteria = vacationData['Cloudiness'] == 0
Wholecriteria = temperatureCriteria & windSpeedCriteria & windSpeedCriteria 

# Use boolean indexing to filter the weather_df dataframe
weather_df = vacationData[Wholecriteria]
weather_df = weather_df.dropna()
weather_df = weather_df.reset_index()
weather_df.head()

,index,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
1,9,9,pushkinskiye gory,6,RU,1558378756,70,57.02,28.91,65.04,6.93
2,12,12,avarua,100,CK,1558378756,94,-21.21,-159.78,71.60,2.24
3,20,20,teknaf,1,BD,1558378758,85,20.86,92.31,84.66,8.23
4,22,22,ambon,11,FR,1558378758,93,47.55,-2.56,64.99,5.82


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#hotel_df = pd.DataFrame(weather_df, columns=['City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Hotel Name'])
#hotel_df

hotel_df = pd.DataFrame(data=vacationData)
hotel_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


In [7]:
#Add a "Hotel Name" column to the DataFrame.

hotel_df.loc[:,'Hotel Name'] = np.nan
hotel_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16,NaN
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41,NaN
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37,NaN
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47,NaN
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05,NaN


In [ ]:
target_radius = 5000
target_type = "hotel"
target_search = "hotel"
hotel_df.loc[:,"Coordinates"] = hotel_df.loc[:,"Lat"].astype(str) +"," + hotel_df.loc[:,"Lng"].astype(str)

hotel_list = []

for location in hotel_df["Coordinates"]:
    params = {
        "location": location,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    gmaps_json = requests.get("https://maps.googleapis.com/maps/api/place/nearbysearch/json", params=params).json()
    
    try:
        hotel_list.append(gmaps_json["results"][0]["name"])
        
    except:
        hotel_list.append("N/A")
        print(f"No hotels within 5 miles of {location}.")

hotel_df["Hotel Name"] = hotel_list
hotel_df.loc[:, ~hotel_df.columns.str.contains('^Unnamed')]


No hotels within 5 miles of 68.05,39.51.
No hotels within 5 miles of 40.71,112.04.
No hotels within 5 miles of 71.97,114.09.
No hotels within 5 miles of 46.84,-105.12.
No hotels within 5 miles of 7.42,99.52.
No hotels within 5 miles of 59.7,150.17.
No hotels within 5 miles of 57.82,114.02.
No hotels within 5 miles of 40.67,33.6.
No hotels within 5 miles of -19.93,-60.77.
No hotels within 5 miles of -34.0,24.73.
No hotels within 5 miles of -25.17,45.06.
No hotels within 5 miles of 13.88,123.7.
No hotels within 5 miles of 39.95,-94.76.
No hotels within 5 miles of 71.98,102.47.
No hotels within 5 miles of 55.31,82.74.
No hotels within 5 miles of 73.51,80.55.
No hotels within 5 miles of 3.07,172.79.
No hotels within 5 miles of 39.45,22.34.
No hotels within 5 miles of 18.9,-3.53.
No hotels within 5 miles of 59.7,30.79.
No hotels within 5 miles of 61.92,63.02.
No hotels within 5 miles of 33.93,134.5.
No hotels within 5 miles of -38.31,-60.23.
No hotels within 5 miles of -1.9,11.91.
No hotels

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(row) for index, row in hotel_df.iterrows()]
)


# Add layer
fig.add_layer(heatmap_layer)
fig.add_layer(hotel_layer)
# Display figure
fig

In [ ]:
"/outputdata/Heatmap.png"